## IRA

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
from pandas.plotting import parallel_coordinates
%matplotlib inline 

In [3]:
df_matriculas = pd.DataFrame()

# importando notas de 2010 a 2017
for i in range(10,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('data_science/matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])

# obtendo o id do curso de ciências e tecnologia
df_cursos = pd.read_csv('data_science/cursos-graduacao.csv', sep=';')
id_cet = df_cursos[df_cursos.nome == 'CIÊNCIAS E TECNOLOGIA'].id_curso
id_cet

20101
20102
20111
20112
20121
20122
20131
20132
20141
20142
20151
20152
20161
20162
20171
20172


17    10320810
Name: id_curso, dtype: int64

In [4]:
# filtra tabela pelo id_curso
df_matriculasCeT = df_matriculas[df_matriculas.id_curso == float(id_cet)]

# columns = ['descricao', 'discente', 'faltas_unidade', 'id_curso', 'id_turma', 'media_final', 'nota', 'numero_total_faltas', 'reposicao', 'unidade']
# df_matriculas = df_matriculas[columns]

In [42]:
aux_teste = df_matriculas[df_matriculas.discente=='72eca9ca3b69e98cb6b5512b3d834913']

In [6]:
df_matriculasCeT.columns

Index(['Unnamed: 10', 'descricao', 'discente', 'faltas_unidade', 'id_curso',
       'id_turma', 'media_final', 'nota', 'numero_total_faltas', 'reposicao',
       'unidade'],
      dtype='object')

In [30]:
df_turmas = pd.read_csv('data_science/turmas-2010-2017-ok.csv')

/home/andressa/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
df_componentes = pd.read_csv('data_science/componentes-curriculares-presenciais-ok.csv')

In [12]:
df_discentes = pd.read_csv('../secret/matriculas-2013-2017.csv', sep=';')

In [13]:
colunas_mat = ['discente', 'id_turma', 'media_final']
df_matriculasCeT = df_matriculasCeT[colunas_mat].dropna().drop_duplicates()

In [14]:
df_matriculasCeT.head(2)

,discente,id_turma,media_final
85637,4746f89573bcb024736f649807753654,57549019,8.6
85640,6ac3d868cc23be038b7b6e70afbd372b,57549019,7.6


In [33]:
df_turmasNome = pd.merge(df_turmas, df_componentes, left_on='id_componente_curricular', right_on='id_componente')

In [35]:
colunas_turma = ['id_turma', 'nome', 'ch_total']
df_turmasNome = df_turmasNome[colunas_turma]
df_turmasNome.head(2)

,id_turma,nome,ch_total
0,1151160,SIMULACAO,60
1,1151161,ESTRATEGIA DE PRODUCAO,60


In [36]:
df_ira = pd.merge(df_matriculasCeT, df_turmasNome, on='id_turma').drop_duplicates()
df_ira.head(2)

,discente,id_turma,media_final,nome,ch_total
0,4746f89573bcb024736f649807753654,57549019,8.6,TÓPICOS ESPECIAIS EM COMPUTAÇÃO XIV,60
2,6ac3d868cc23be038b7b6e70afbd372b,57549019,7.6,TÓPICOS ESPECIAIS EM COMPUTAÇÃO XIV,60


In [37]:
df_ira['notaXch'] = df_ira['media_final']*df_ira['ch_total']

In [38]:
df_sumNotaCh = df_ira.groupby('discente').notaXch.sum().reset_index()
display(df_sumNotaCh.head(2))

df_sumCh = df_ira.groupby('discente').ch_total.sum().reset_index()
display(df_sumCh.head(2))

,discente,notaXch
0,0004fd054c6915cbd14905a92c38d40a,1749.0
1,0010345ff1be7da25478b5a96c772616,8703.0


,discente,ch_total
0,0004fd054c6915cbd14905a92c38d40a,610
1,0010345ff1be7da25478b5a96c772616,1710


In [39]:
df_calIra = pd.merge(df_sumNotaCh, df_sumCh, on='discente')
df_calIra['ira'] = df_calIra['notaXch']/df_calIra['ch_total']

df_calIra.head(2)

,discente,notaXch,ch_total,ira
0,0004fd054c6915cbd14905a92c38d40a,1749.0,610,2.867213
1,0010345ff1be7da25478b5a96c772616,8703.0,1710,5.089474


In [41]:
df_calIra[df_calIra['discente'] == '72eca9ca3b69e98cb6b5512b3d834913']

,discente,notaXch,ch_total,ira
4796,72eca9ca3b69e98cb6b5512b3d834913,16732.0,2410,6.942739
